# Integrator

## Euler method

- is a first-order numerical procedure for solving ordinary differential equations (ODEs) with a given initial value
- approximating the integral with the finite sum 
- by the Taylor expansion, the quadratic and higher-order terms are ignored

Approximation 
$$ x(t+\tau)=x(t)+\tau\, v(t)\\
v(t+\tau)=v(t)+\tau\, a(t)=v(t)-\frac{\tau}{m}\,\nabla\phi(x(t)) \\
\text{with}\,\, v= \frac{\partial x}{\partial t}; \,\, \text{and}\,\, a=\frac{F}{m}=-\frac{1}{m}\nabla \phi(x(t))$$

### Goals
- setup of the algorithm 
    - parameters  
    - inputs
- timing
- 1D, 2D and 3D distinction
- vectorization
- testing
    - ZeroDivisionError, NameError and TypeErrorn ValueError, Type,Error, RuntimeError, IOError
    - functionality
    - ??
- visualisation
    - timing
    - stability
    - ??

### setup of the algorithm
- import packages
- define function with its inputs
    - (function): potential gradient 
    - (parameter): mass
    - (functionality): size (number of time steps), time step, initial configuration, initial velocity
- creating positiion and velocity container (vector, array, np. zeros ??)
- initialization by adding the start configuration
- time iteration 
    - euler approximation
    - write in pos and vel container
- returning position and velocety container


In [ ]:
%matplotlib notebook
import numpy as np
import timeit
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
# wehmeyer
def eulerW(potential_gradient, size=100, x_init=1.0, v_init=0.0, mass=1.0, time_step=0.005):
    x, v = np.zeros(size), np.zeros(size)
    x[0], v[0] = x_init, v_init
    for i in range(1, size):
        x[i] = x[i - 1] + time_step * v[i - 1]
        v[i] = v[i - 1] - time_step * potential_gradient(x[i - 1]) / mass
    return x, v

def potential_gradient(x):
    return x

#timeit.timeit(eulerW, number= 100) # geht nicht warum?

#### open system

In [ ]:
def potential_gradient(p, k=10):
    return k*p

In [ ]:
def eulerJ(potential_gradient, position_init, velocity_init, mass, T, time_step):
    size = int(T/time_step)  # number of time steps
    n = len(position_init)   # number of particles
    dim = position_init.shape[-1]
    m = mass
    #print(n)
    # creating positiion and velocity container
    position_matrix, velocity_matrix, acceleration_matrix = np.zeros((size, n, dim)), np.zeros((size, n, dim)), np.zeros((size, n, dim))
    # initialization by adding the start configuration
    position_matrix[0], velocity_matrix[0], acceleration_matrix[0] = position_init, velocity_init, - 1/m *potential_gradient(position_init)
    # time iteration
    for t in range(1, size):
        # rename # kostet das rechenleistung ? bennenung lieber von anfang anders?
        p = position_matrix[t-1]
        v = velocity_matrix[t-1]
        a = acceleration_matrix[t]
        gp = potential_gradient(p)
        # euler approximation
        p_new = p + time_step*v
        #print(v,gp)
        v_new = v - time_step/m * gp
        a = - potential_gradient(p_new)/m
        # write in pos and vel container
        position_matrix[t], velocity_matrix[t], acceleration_matrix[t] = p_new, v_new, a
    # returning position and velocety container
    return position_matrix, velocity_matrix, acceleration_matrix

#%line_profiler

In [ ]:
p_0 = np.array([[1., 0., 0.], [0., 0., 0.]])
v_0 = np.array([[100., 100., 100.], [100., 100., 100.]])
m = np.array([[1., 1., 1.], [1., 1., 1.]])

res = eulerJ(potential_gradient, p_0, v_0, m, 10, 1/100)
res
#print(a[0][:,0,0])
plt.plot(res[0][:,0, 0], res[0][:,0,1])

### closed system

In [ ]:
def potential_gradient(p, k=1):
    p_max_r = p_max - p_max/4
    p_max_l = p_max/4
    V_max_r = 4 * np.abs(p - p_max_r)**2*np.heaviside(p, np.abs(p - p_max_r))
    V_max_l = 4 * np.abs(p - p_max_l)**2*np.heaviside(p, np.abs(p - p_max_l))
    p_min_r = p_min - p_min/4
    p_min_l = p_min/4
    V_min_r = 4 * np.abs(p - p_min_r)**2*np.heaviside(p, np.abs(p - p_min_r))
    V_min_l = 4 * np.abs(p - p_min_l)**2*np.heaviside(p, np.abs(p - p_min_l))
    V = k*p
    return V * V_max_l * V_max_r * V_min_l * V_min_r

#def box_potential(p, p_max):
    #p_r = p_max - p_max/4
    #p_l = p_max/4
    #V_r = 4 * np.abs(p - p_r)**2*np.heaviside(p, np.abs(p - p_l))
    #V_l = 4 * np.abs(p - p_l)**2*np.heaviside(p, np.abs(p - p_l))
    #return V_r + V_l



#def potential_gradient(gradient, box_potential):
    #return gradient + box_potential

In [ ]:
def potential_gradient(p, p_max=5, p_min=0, k=10):
    V = k*p
    V_end = 0
    if np.any(p >= p_max):
        V_r = 5 * np.abs(p - (p_max - p_max/4)**2*np.any(p <= (p_max - p_max/4))
        V_end = V_end + V_r
        if np.any(p <= p_min):
            V_l = 5 * np.abs(p - p_max/4)**2*np.any(p >= p_max/4)
            V_end = V_end * V_l
            V += V_end
            print(V)
    return V  
p = np.array([[5],[0],[1],[4]])
pot = potential_gradient(p)
#plt.plot(p, pot)

In [ ]:
def potential_gradient(p, p_max=5, p_min=0, k=1):
    V = k*p
    V_r = 2*np.abs(p*(p>=p_max) - (p_max - p_max/4)*(p>=p_max))**2
    V_l = 2*np.abs(p*(p<=p_min) - (p_max/4)*(p<=p_min))**2
    V = V+ V_r+ V_l 
    #print(V_l)
    #print(V_r)
    #print(V)
    return V  
#p = np.array([[5, 0, 1],[0, 1, 6],[1, 6, 0],[4, 1, 3]])
#pot = potential_gradient(p)
#plt.plot(p[:, 1], pot[:,1])

In [ ]:
def eulerclose(potential_gradient, position_init, velocity_init, mass, T, time_step):
    size = int(T/time_step)  # number of time steps
    n = len(position_init)   # number of particles
    dim = position_init.shape[-1]
    m = mass
    #print(n)
    # creating positiion and velocity container
    position_matrix, velocity_matrix, acceleration_matrix = np.zeros((size, n, dim)), np.zeros((size, n, dim)), np.zeros((size, n, dim))
    # initialization by adding the start configuration
    position_matrix[0], velocity_matrix[0], acceleration_matrix[0] = position_init, velocity_init, - 1/m *potential_gradient(position_init)
    # time iteration
    for t in range(1, size):
        # rename # kostet das rechenleistung ? bennenung lieber von anfang anders?
        p = position_matrix[t-1]
        v = velocity_matrix[t-1]
        a = acceleration_matrix[t]
        gp = potential_gradient(p)
        # euler approximation
        p_new = p + time_step*v
        v_new = v - time_step/m * gp
        a = - potential_gradient(p_new)/m
        # write in pos and vel container
        position_matrix[t], velocity_matrix[t], acceleration_matrix[t] = p_new, v_new, a
    # returning position and velocety container
    return position_matrix, velocity_matrix, acceleration_matrix

In [ ]:
# testing 
# input 2D
p_0 = np.array([[1., 1.]])
v_0 = np.array([[0., 0.]])
p_max = np.array([[5., 5.]])
p_min = np.array([[0., 0.]])
#p_max = 5.
m = np.array([[1., 1.]])
# input 3D
#p_max = np.array([[2., 3., 2.], [2., 3., 2.]])
#p_0 = np.array([[1., 0., 0.], [0., 1., 0.]])
#v_0 = np.array([[0., 0., 0.], [0., 0., 0.]])
#m = np.array([[1., 1., 1.], [1., 1., 1.]])

# result
res = eulerclose(potential_gradient, p_0, v_0, m, 100, 1/10)
res
#print(a[0][:,0,0])
plt.plot(res[0][:,0, 0], res[1][:,0,0])

# potential
#pot = potential_gradient(p_0)
#plt.plot(res[2][:, 0, 1], res[0][:,0, 1])

### periodic system

In [ ]:
def potential_gradient(p,k=10):
    return k*p

In [ ]:
def eulerpbc(potential_gradient, position_init, velocity_init, mass, T, time_step, pbc=True):
    size = int(T/time_step)  # number of time steps
    n = len(position_init)   # number of particles
    dim = position_init.shape[-1]
    m = mass
    #print(n)
    # creating positiion and velocity container
    position_matrix, velocity_matrix, acceleration_matrix = np.zeros((size, n, dim)), np.zeros((size, n, dim)), np.zeros((size, n, dim))
    # initialization by adding the start configuration
    position_matrix[0], velocity_matrix[0], acceleration_matrix[0] = position_init, velocity_init, - 1/m *potential_gradient(position_init)
    # time iteration
    for t in range(1, size):
        # rename # kostet das rechenleistung ? bennenung lieber von anfang anders?
        p = position_matrix[t-1]
        v = velocity_matrix[t-1]
        a = acceleration_matrix[t]
        gp = potential_gradient(p)
        # euler approximation
        p_new = p + time_step*v
        if pbc == True:
            p_new = abs(abs(p_new) - (np.any(p_new >= p_max) or np.any(p_new <= p_min))*p_max)
        v_new = v - time_step/m * gp
        a = - potential_gradient(p_new)/m
        # write in pos and vel container
        position_matrix[t], velocity_matrix[t], acceleration_matrix[t] = p_new, v_new, a
    # returning position and velocety container
    return position_matrix, velocity_matrix, acceleration_matrix

In [ ]:
# testing 
# input 2D
#p_0 = np.array([[1., 0.]])
#v_0 = np.array([[1., 0.]])
p_max = 5.
p_min = 0.
#m = np.array([[1., 1.]])
# input 3D
#p_max = np.array([[2., 3., 2.], [2., 3., 2.]])
p_0 = np.array([[5., 0., 0.], [0., 1., 0.]])
v_0 = np.array([[0., 0., 0.], [0., 0., 0.]])
m = np.array([[1., 1., 1.], [1., 1., 1.]])

# result
res = eulerpbc(potential_gradient, p_0, v_0, m, 100, 1/100)
#res
#print(a[0][:,0,0])
plt.plot(res[0][:,0, 0], res[1][:,0,0])
#print(res[0])

### all combined

In [ ]:
M = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
    ])
print(M)
print(type(M))
print(type(M).__module__)

In [ ]:
def box_potential(p, p_max):
    p_r = p_max - p_max/4
    print(p_r)
    p_l = p_max/4
    print(p_l)
    print(p - p_r)
    V_r = 4 * np.abs(p - p_r)**2*np.heaviside(p, np.abs(p - p_l))
    V_l = 4 * np.abs(p - p_l)**2*np.heaviside(p, np.abs(p - p_l))
    return np.array(V_r + V_l)

p = np.array([[1., 0.]])
p_max = np.array([[5., 5.]])
#p_max = 5.
res = box_potential(p, p_max)